In [2]:
%%file submissions/starting_kit/image_classifier.py
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.optimizers import Adam
from rampwf.workflows.image_classifier import get_nb_minibatches
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


class ImageClassifier(object):

    def __init__(self):
        self.batch_size = 5
        self.img_width, self.img_height = 32, 32
        self.model = Sequential()
        self._build_model()
    
    def _build_model(self):
        
        # Forme de l'input en fonction du backend
        if K.image_data_format() == 'channels_first':
            self.input_shape = (3, self.img_width, self.img_height)
        else:
            self.input_shape = (self.img_width, self.img_height, 3)

        self.model.add(Conv2D(32, (3, 3), input_shape=self.input_shape))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(64, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        
        self.model.add(Conv2D(64, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Flatten())
        self.model.add(Dense(64))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        
        # Compute the probabilities of the ten classes
        self.model.add(Dense(10)) 
        self.model.add(Activation('softmax'))

        self.model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])

    def _transform(self, x):
        x = resize(x, (self.img_width, self.img_height), preserve_range=True, anti_aliasing=False)
        # bringing input between 0 and 1
        x = x / 255.
        return x

    def _build_train_generator(self, img_loader, indices, batch_size,
                               shuffle=False):
        indices = indices.copy()
        nb = len(indices)
        X = np.zeros((batch_size, self.img_width, self.img_height, 3))
        Y = np.zeros((batch_size, 10))
        while True:
            if shuffle:
                np.random.shuffle(indices)
            for start in range(0, nb, batch_size):
                stop = min(start + batch_size, nb)
                # load the next minibatch in memory.
                # The size of the minibatch is (stop - start),
                # which is `batch_size` for the all except the last
                # minibatch, which can either be `batch_size` if
                # `nb` is a multiple of `batch_size`, or `nb % batch_size`.
                bs = stop - start
                Y[:] = 0
                for i, img_index in enumerate(indices[start:stop]):
                    x, y = img_loader.load(img_index)
                    print(x.shape)
                    if (x.shape)==(1163, 1600, 3): #(1536, 2048, 3):
                        plt.imshow(x)
                     # TODO TEMP
                    if len(x.shape)!=3 or x.shape[2]!=3 or x.shape[1]==1200:
                        print('passed')
                        Y[i, 0] = 1
                        continue
                    x = self._transform(x)
                    X[i] = x
                    Y[i, y] = 1
                yield X[:bs], Y[:bs]

    def _build_test_generator(self, img_loader, batch_size):
        nb = len(img_loader)
        X = np.zeros((batch_size, self.img_width, self.img_height, 3))
        while True:
            for start in range(0, nb, batch_size):
                stop = min(start + batch_size, nb)
                # load the next minibatch in memory.
                # The size of the minibatch is (stop - start),
                # which is `batch_size` for the all except the last
                # minibatch, which can either be `batch_size` if
                # `nb` is a multiple of `batch_size`, or `nb % batch_size`.
                bs = stop - start
                for i, img_index in enumerate(range(start, stop)):
                    x = img_loader.load(img_index)
                    x = self._transform(x)
                    X[i] = x
                yield X[:bs]

    def fit(self, img_loader):
        np.random.seed(24)
        nb = len(img_loader)
        nb_train = int(nb * 0.9)
        nb_valid = nb - nb_train
        indices = np.arange(nb)
        np.random.shuffle(indices)
        ind_train = indices[0: nb_train]
        ind_valid = indices[nb_train:]

        gen_train = self._build_train_generator(
            img_loader,
            indices=ind_train,
            batch_size=self.batch_size,
            shuffle=True
        )
        gen_valid = self._build_train_generator(
            img_loader,
            indices=ind_valid,
            batch_size=self.batch_size,
            shuffle=True
        )
        self.model.fit_generator(
            gen_train,
            steps_per_epoch=get_nb_minibatches(nb_train, self.batch_size),
            epochs=1,
            max_queue_size=16,
            workers=1,
            use_multiprocessing=True,
            validation_data=gen_valid,
            validation_steps=get_nb_minibatches(nb_valid, self.batch_size),
            verbose=1
        )

    def predict_proba(self, img_loader):
        nb_test = len(img_loader)
        gen_test = self._build_test_generator(img_loader, self.batch_size)
        return self.model.predict_generator(
            gen_test,
            steps=get_nb_minibatches(nb_test, self.batch_size),
            max_queue_size=16,
            workers=1,
            use_multiprocessing=True,
            verbose=0
        )

Overwriting submissions/starting_kit/image_classifier.py


In [ ]:
!ramp_test_submission

Testing Fruits and vegetables classification (10 classes)
Reading train and test files from ./data ...
Reading cv ...
Training submissions/starting_kit ...
CV fold 0
Using TensorFlow backend.
Epoch 1/1
(2100, 650, 3)
/home/alexandre/Projects/master/datacamp/collab/clbenv/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
(1536, 2048, 3)
/home/alexandre/Projects/master/datacamp/collab/clbenv/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
(1024, 768, 3)
(278, 278, 3)
(750, 1000, 3)
(675, 948, 4)
passed
(533, 800, 3)
(1365, 2048, 3)
(1226, 800, 3)
(2215, 3478, 3)
(844, 1500, 3)
(1125, 1688, 3)
(610, 570, 3)
(1198, 1200, 3)
passed
2019-01-30 20:04:05

(640, 859, 3)
(685, 1024, 3)
(2048, 1365, 3)
(2000, 3008, 3)
 33/859 [>.............................] - ETA: 18:23 - loss: 2.2875 - acc: 0.1636(2304, 3072, 3)
(265, 320, 3)
(443, 640, 3)
(563, 612, 3)
(667, 1000, 3)
 34/859 [>.............................] - ETA: 18:05 - loss: 2.2957 - acc: 0.1588(1000, 1000, 3)
(823, 1024, 3)
(1480, 4260, 3)
(320, 450, 3)
(1536, 2048, 3)
 35/859 [>.............................] - ETA: 17:57 - loss: 2.2943 - acc: 0.1600(1200, 1600, 3)
(4288, 2848, 3)
(878, 1500, 4)
passed
(1600, 1201, 3)
(363, 600, 4)
passed
 36/859 [>.............................] - ETA: 18:02 - loss: 2.2880 - acc: 0.1667(3648, 2736, 3)
(3346, 2530, 3)
(1423, 2048, 3)
(467, 700, 3)
(580, 600, 4)
passed
 37/859 [>.............................] - ETA: 18:07 - loss: 2.2866 - acc: 0.1676(683, 1024, 3)
(960, 1280, 3)
(360, 460, 3)
(800, 800, 3)
(2220, 1944, 3)
(255, 279, 3)
 38/859 [>.............................] - ETA: 17:52 - loss: 2.2974 - acc: 0.1632(768, 1024, 3)
(335, 500, 3)
(1024,

In [4]:
%matplotlib inline